Custom datasets file 위치 : '/home/sldev1/Project/hyeongeun_test/data'   
먼저 할 것은, tar를 시각적으로 어케보는지 확인해보자.
###### 완료
-----
라벨링 어떻게 되어있는지 데이터 확인하자.   
10/26일 할일 annotation 어케 되어있는지 확인하자.

## tar Data Visualization (22.10.25)

In [2]:
import tarfile

In [6]:
with tarfile.open('/home/sldev1/Project/hyeongeun_test/data/train_set.tar', 'r') as tar_file:
    print(tar_file.getnames())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
tar = tarfile.open('/home/sldev1/Project/hyeongeun_test/data/train_set.tar', mode='r')

In [29]:
type(tar)
list(tar)
Image(tar
     )

TypeError: 'module' object is not callable

In [16]:
import torch.utils.data as data
import os
import re
import torch
import tarfile
from PIL import Image


IMG_EXTENSIONS = ['.png', '.jpg', '.jpeg']


def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_.lower())]


def _extract_tar_info(tarfile):
    class_to_idx = {}
    files = []
    targets = []
    for ti in tarfile.getmembers():
        if not ti.isfile():
            continue
        dirname, basename = os.path.split(ti.path)
        target = os.path.basename(dirname)
        class_to_idx[target] = None
        ext = os.path.splitext(basename)[1]
        if ext.lower() in IMG_EXTENSIONS:
            files.append(ti)
            targets.append(target)
    for idx, c in enumerate(sorted(class_to_idx.keys(), key=natural_key)):
        class_to_idx[c] = idx
    tarinfo_and_targets = zip(files, [class_to_idx[t] for t in targets])
    tarinfo_and_targets = sorted(tarinfo_and_targets, key=lambda k: natural_key(k[0].path))
    return tarinfo_and_targets


class ImageFolderTar(data.Dataset):

    def __init__(self, root, transform=None):

        assert os.path.isfile(root)
        self.root = root
        with tarfile.open(root) as tf:  # cannot keep this open across processes, reopen later
            self.imgs = _extract_tar_info(tf)
        self.tarfile = None  # lazy init in __getitem__
        self.transform = transform

    def __getitem__(self, index):
        if self.tarfile is None:
            self.tarfile = tarfile.open(self.root)
        tarinfo, target = self.imgs[index]
        iob = self.tarfile.extractfile(tarinfo)
        img = Image.open(iob).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgs)

In [19]:
test1 = ImageFolderTar(root='/home/sldev1/Project/hyeongeun_test/data/train_set.tar')

In [50]:
import torchvision.transforms as tr

transf = tr.Compose(tr.RandomCrop(32, padding=4))
test2 = ImageFolderTar(root='/home/sldev1/Project/hyeongeun_test/data/train_set.tar', trainsform=transf)

In [87]:
#test2[0].tarinfo
img, target = test1[5]
img.show()


#test1[0]

## Data Annotaion Check (2022.10.26)
#### 1. npy파일 타입의 이점(insight 용, 결론 : better than csv, txt)   
https://towardsdatascience.com/what-is-npy-files-and-why-you-should-use-them-603373c78883   
   
#### 2. 상준 선임님께 img+npy 합쳐서 못보냐고 물어봤는데, 답은 여기는 없다고 하심.   
고로, Data Annotation Check는 npy 염탐하는 부분에서 마무리 짓겠음.

In [34]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### npy file 염탐하기

In [47]:
data_array = np.load('/home/sldev1/Project/hyeongeun_test/data/FER/train_set/annotations/100000_aro.npy')
print(data_array)

-0.137627


In [48]:
data_array = np.load('/home/sldev1/Project/hyeongeun_test/data/FER/train_set/annotations/100000_exp.npy')
print(data_array)

1


##### 위에 정수 값 별 의미
0 : Neutral   
1 : Happiness   
2 : Sadness   
3 : Suprise   
4 : Fear   
5 : Disgust   
6 : Anger   
7 : Content   

In [49]:
data_array = np.load('/home/sldev1/Project/hyeongeun_test/data/FER/train_set/annotations/100000_lnd.npy')
print(data_array)

[ -3.66446602 108.67054545  -2.27262136 132.00727273   0.45669903
 153.72509091   9.07961165 176.288       22.04116505 195.08363636
  40.73320388 210.27490909  66.85203883 220.70109091  92.18796117
 228.24581818 116.28427184 228.94836364 136.38990291 222.77818182
 151.68932039 206.95563636 166.72776699 190.4        176.52504854
 171.88945455 181.61398058 152.17745455 186.05048544 130.97890909
 188.24699029 112.90618182 186.77902913  93.968       20.88854369
  73.07490909  37.38407767  64.45090909  54.68427184  60.11345455
  73.92        60.79563636  90.61126214  69.17527273 126.37514563
  66.80290909 140.79378641  57.46618182 156.27805825  52.96581818
 171.81669903  54.19781818 183.4407767   63.53454545 110.41242718
  86.352      113.63106796 102.24581818 117.31728155 117.42690909
 120.69902913 133.25963636  98.37514563 147.20872727 108.17242718
 148.94981818 117.26291262 150.79272727 125.38563107 147.392
 132.51883495 142.61672727  42.73398058  93.58109091  53.82524272
  85.12        

In [50]:
data_array = np.load('/home/sldev1/Project/hyeongeun_test/data/FER/train_set/annotations/100000_val.npy')
print(data_array)

0.784476


In [16]:
print(len(img_list))
print(len(annot_list)) #img : annot = 1 : 4

3999
15996


In [17]:
print(img_list[:10])
print(annot_list[:10])

['/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/0.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/10.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/100.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1001.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1002.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1003.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1007.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1008.jpg', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/images/1009.jpg']
['/home/sldev1/Project/hyeongeun_test/data/FER/val_set/annotations/0_aro.npy', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/annotations/0_exp.npy', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set/annotations/0_lnd.npy', '/home/sldev1/Project/hyeongeun_test/data/FER/val_set

In [18]:
print(3999*4)

15996


## npy Custom Data training with pytorch (2022.10.26)
#### 1. baseline code
https://freshrimpsushi.github.io/posts/how-to-create-and-use-custom-data-set-from-numpy-arrays-in-pytorch/   
위에꺼 일단 보류.   
밥먹고나서 데이터불러오기_커스터마이징 참고해서 이 파일 위에 tar visual부분이랑 결합해서 해보자.   
일단 1개 npy부터 해보자.
   

In [1]:
import torch.utils.data as data
import os
import re
import torch
import tarfile
from PIL import Image


IMG_EXTENSIONS = ['.png', '.jpg', '.jpeg']


def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_.lower())]


def _extract_tar_info(tarfile):
    class_to_idx = {}
    files = []
    targets = []
    for ti in tarfile.getmembers():
        if not ti.isfile():
            continue
        dirname, basename = os.path.split(ti.path)
        target = os.path.basename(dirname)
        class_to_idx[target] = None
        ext = os.path.splitext(basename)[1]
        if ext.lower() in IMG_EXTENSIONS:
            files.append(ti)
            targets.append(target)
    for idx, c in enumerate(sorted(class_to_idx.keys(), key=natural_key)):
        class_to_idx[c] = idx
    tarinfo_and_targets = zip(files, [class_to_idx[t] for t in targets])
    tarinfo_and_targets = sorted(tarinfo_and_targets, key=lambda k: natural_key(k[0].path))
    return tarinfo_and_targets


class ImageFolderTar(data.Dataset):

    def __init__(self, root, transform=None):

        assert os.path.isfile(root)
        self.root = root
        with tarfile.open(root) as tf:  # cannot keep this open across processes, reopen later
            self.imgs = _extract_tar_info(tf)
        self.tarfile = None  # lazy init in __getitem__
        self.transform = transform

    def __getitem__(self, index):
        if self.tarfile is None:
            self.tarfile = tarfile.open(self.root)
        tarinfo, target = self.imgs[index]
        iob = self.tarfile.extractfile(tarinfo)
        img = Image.open(iob).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgs)

In [2]:
test1 = ImageFolderTar(root='/home/sldev1/Project/hyeongeun_test/data/train_set.tar')

In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as tr

In [37]:
transform_train = tr.Compose([tr.RandomCrop(32, padding=4), 
                              tr.ToTensor(),
                              tr.Normalize])
transform_test = tr.Compose([tr.ToTensor()])


train_dataset = ImageFolderTar(root='/home/sldev1/Project/hyeongeun_test/data/train_set.', transform=transform_train)
test_dataset = ImageFolderTar(root='/home/sldev1/Project/hyeongeun_test/data/val_set.tar', transform=transform_test)

In [44]:
img, target = train_dataset[3]
print(img.shape)
print(len(train_dataset))

torch.Size([3, 32, 32])
287651


In [55]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [56]:
len(train_loader)

2248

In [57]:
#num_workers=2이상으로 설정하니까 Error나네?
#num_workers 설정 안하면 Error안남.
dataiter = iter(train_loader)
images, labels = dataiter.next()

OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-1-5849c200a3a0>", line 54, in __getitem__
    img = Image.open(iob).convert('RGB')
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 915, in convert
    self.load()
  File "/usr/local/lib/python3.6/dist-packages/PIL/ImageFile.py", line 274, in load
    raise_oserror(err_code)
  File "/usr/local/lib/python3.6/dist-packages/PIL/ImageFile.py", line 67, in raise_oserror
    raise OSErr

In [58]:
images.size()

torch.Size([128, 3, 32, 32])

customdata를 data load까지 전처리 완료!!

이제 학습시켜볼게

In [59]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from resnet import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
import os
import torchvision.models as models

''' random seed fixed'''
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [60]:
#Simple Learning Rate Scheduler
def lr_scheduler(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr /= 10
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Xavier
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [61]:
device = 'cuda'
model = ResNet50()
# ResNet18, ResNet34, ResNet50, ResNet101, ResNet152 중에 택일하여 사용

In [62]:
model.apply(init_weights)
model = model.to(device)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [63]:
learning_rate = 0.1
num_epoch = 150
model_name = 'model.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

에러남 ㅠ

In [64]:
# Train
for epoch in range(num_epoch):
    print(f"====== { epoch+1} epoch of { num_epoch } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0
    # Train Phase
    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        
        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)
        
        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()
        
        if step % 100 == 0 and step != 0:
            print(f"\n====== { step } Step of { len(train_loader) } ======")
            print(f"Train Acc : { correct / total_cnt }")
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")
            
    correct = 0
    total_cnt = 0
    
# Test Phase
    with torch.no_grad():
        model.eval()
        for step, batch in enumerate(test_loader):
            # input and target
            batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
            total_cnt += batch[1].size(0)
            logits = model(batch[0])
            valid_loss += loss_fn(logits, batch[1])
            _, predict = logits.max(1)
            correct += predict.eq(batch[1]).sum().item()
        valid_acc = correct / total_cnt
        print(f"\nValid Acc : { valid_acc }")    
        print(f"Valid Loss : { valid_loss / total_cnt }")

        if(valid_acc > best_acc):
            best_acc = valid_acc
            torch.save(model, model_name)
            print("Model Saved!")

====== 1 epoch of 150 ======


UnidentifiedImageError: Caught UnidentifiedImageError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-1-5849c200a3a0>", line 54, in __getitem__
    img = Image.open(iob).convert('RGB')
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 3031, in open
    "cannot identify image file %r" % (filename if filename else fp)
PIL.UnidentifiedImageError: cannot identify image file <ExFileObject name='/home/sldev1/Project/hyeongeun_test/data/train_set.tar'>


In [ ]:
'''
상준 선임님 왈:
1. 파싱 먼저하고
2. function 만들어 준다.
'''

###난이도 너무 높아졌는데 ???????????????/ 집간다.

In [7]:
EMOTIONS = [{"Neutral"   : "0",
            "Happiness" : "1",
            "Sadness"   : "2", 
            "Suprise"   : "3", 
            "Fear"      : "4", 
            "Disgust"   : "5", 
            "Anger"     : "6",
            "Content"   : "7"}]